# 🎬 Movie Recommendation System Demo

This notebook demonstrates how to use the content-based movie recommender system **with data validation checks**.

In [ ]:
!pip install -r ../requirements.txt

## 📥 Load the Recommender (With Validation)

In [ ]:
from src.movies import MovieRecommendationSystem
import pandas as pd

# ---- Data Loading with Validation ----
def validate_data(movies_path, credits_path):
    """Checks for missing values, duplicates, and data types."""
    try:
        movies = pd.read_csv(movies_path)
        credits = pd.read_csv(credits_path)
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Data file missing: {e}")

    # Check for missing values
    for df, name in [(movies, "Movies"), (credits, "Credits")]:
        print(f"\n{name} data - Missing values:")
        print(df.isnull().sum())
        df.dropna(subset=['title'], inplace=True)  # Critical field

    # Check for duplicates
    if movies.duplicated(subset=['title']).any():
        print("\n⚠ Warning: Duplicate movie titles found. Dropping duplicates.")
        movies.drop_duplicates(subset=['title'], inplace=True)

    return movies, credits

# Load data with validation
movies_path = '../data/tmdb_5000_movies.csv'
credits_path = '../data/tmdb_5000_credits.csv'
movies, credits = validate_data(movies_path, credits_path)

# Initialize recommender
recommender = MovieRecommendationSystem(movies_path, credits_path)
recommender.load_and_preprocess_data()
recommender.build_similarity_matrix()

## 🎯 Get Recommendations

In [ ]:
movie = 'Avatar'
try:
    results = recommender.get_recommendations(movie)
    if not results['recommendations']:
        print(f"\n❌ No recommendations found for '{movie}'. Is the title correct?")
    else:
        print(f"\n🎬 Recommendations for '{movie}':")
        for i, rec in enumerate(results['recommendations'], 1):
            print(f"{i}. {rec['title']} (Similarity: {rec['similarity_score']:.2f})")
except KeyError:
    print(f"\n❌ Movie '{movie}' not found in dataset. Check spelling or try another title.")